In [1]:
"""
-Arman Bazarchi-
Multimodal Embeddings with CLIP

This notebook generates multimodal embeddings for our data by combining image and text 
information into a shared embedding space.

Key steps:
    Connects to a MinIO bucket to load metadata and images from trusted-zone.
    Processes metadata to create a simple natural descriptions for each species.
    Uses OpenCLIP to generate embeddings for both images and text.
    Stores embeddings in ChromaDB, ensuring no duplicates are added.


"""

import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
import io
import os
import re
from minio import Minio
import sys
from PIL import Image
import numpy as np
from datetime import datetime

# ==============================
#          Functions
# ==============================

def clean_value(value):
    # Clean up metadata values by removing invalid entries.
    return value if value and value != 'nan' and value != 'None' and value != '' else ''

def create_combined_text(row):
    # Create enriched combined text with natural descriptions from metadata.
    # Extract key fields
    
    common_name = clean_value(str(row.get('common', '')).strip())
    scientific_name = clean_value(str(row.get('scientific_name', '')).strip())
    family = clean_value(str(row.get('family', '')).strip())
    genus = clean_value(str(row.get('genus', '')).strip())
    species = clean_value(str(row.get('species', '')).strip())
    kingdom = clean_value(str(row.get('kingdom', '')).strip())
    phylum = clean_value(str(row.get('phylum', '')).strip())
    class_name = clean_value(str(row.get('class', '')).strip())
    order = clean_value(str(row.get('order', '')).strip())
    
    # Build natural description 
    description_parts = ["This life is"]
    
    # Add common name if available
    if common_name:
        description_parts.append(f"commonly known as {common_name}")
    elif scientific_name:
        description_parts.append(f"scientifically named as {scientific_name}")
    else:
        description_parts.append("wildlife species")
    
    # Add scientific name if different from common name
    if scientific_name and scientific_name != common_name:
        description_parts.append(f"scientifically named as {scientific_name}")
    
    # Add all taxonomic details
    if kingdom:
        description_parts.append(f"in kingdom {kingdom}")
    if phylum:
        description_parts.append(f"phylum {phylum}")
    if class_name:
        description_parts.append(f"in the {class_name} class")
    if order:
        description_parts.append(f"order of {order}")
    if family:
        description_parts.append(f"belonging to the {family} family")
    if genus:
        description_parts.append(f"genus {genus}")
    if species:
        description_parts.append(f"and species of {species}")
    
    return ' '.join(description_parts) + '.'

def ensure_native_str(value):
    # Ensure native Python strings for ChromaDB compatibility.
    if isinstance(value, str):
        return value
    if value is pd.NA or pd.isna(value):
        return ""
    try:
        # unwrap pandas or numpy scalars
        if hasattr(value, "item"):
            value = value.item()
        return str(value)
    except Exception:
        return str(value)

def setup_minio_client(minio_endpoint, access_key, secret_key, trusted_bucket):
    # Setup and verify MinIO client connection.
    
    print(" Connecting to MinIO...")
    client = Minio(minio_endpoint, access_key=access_key, secret_key=secret_key, secure=False)
    
    if not client.bucket_exists(trusted_bucket):
        sys.exit(f" ERROR: Bucket '{trusted_bucket}' does not exist in MinIO.")
    
    print(f" Connected to MinIO - Bucket '{trusted_bucket}' verified")
    return client

def setup_chromadb_collection(chroma_db_path, collection_name):
    # Setup ChromaDB client and collection.
    print(" Setting up ChromaDB...")
    chroma_client = chromadb.PersistentClient(path=chroma_db_path)
    clip_ef = embedding_functions.OpenCLIPEmbeddingFunction()
    
    collection = chroma_client.get_or_create_collection(
        name=collection_name,
        embedding_function=clip_ef,
        metadata={"description": "Multimodal wildlife embeddings with images and text"})
    
    return collection

def load_metadata_from_minio(client, trusted_bucket):
    # Load and process metadata from MinIO trusted zone.
    print(" Loading metadata from trusted-zone...")
    
    # Find the latest metadata file with timestamp
    trusted_meta_prefix = "metadata/"
    metadata_objs = [
        obj.object_name for obj in client.list_objects(trusted_bucket, prefix=trusted_meta_prefix, recursive=True)
        if obj.object_name.lower().endswith(".csv")
    ]
    
    if not metadata_objs:
        sys.exit(" ERROR: No metadata CSV files found in trusted-zone/metadata/")
    
    # Get the latest metadata file
    latest_metadata = sorted(metadata_objs)[-1]
    print(f" Found {len(metadata_objs)} metadata files, using latest: {latest_metadata}")
    
    local_metadata = "temp_metadata.csv"
    
    try:
        client.fget_object(trusted_bucket, latest_metadata, local_metadata)
        print(f" Downloaded metadata from {latest_metadata}")
    except Exception as e:
        sys.exit(f" ERROR: Failed to download metadata → {e}")
    
    # Load metadata
    metadata_df = pd.read_csv(local_metadata)
    print(f" Loaded {len(metadata_df)} records from metadata")
    
    # Create enriched combined_text
    metadata_df['combined_text'] = metadata_df.apply(create_combined_text, axis=1)
    
    # Clean up local file
    os.remove(local_metadata)
    print(" Cleaned up temporary metadata file")
    print("=" * 50)
    
    return metadata_df

def get_existing_embeddings(collection):
    # Get existing embedding IDs to avoid duplicates.
    existing_count = collection.count()
    existing_ids = []
    
    if existing_count > 0:
        batch_size = 500
        offset = 0
        while True:
            batch = collection.get(limit=batch_size, offset=offset)
            if not batch["ids"]:
                break
            existing_ids.extend(batch["ids"])
            offset += batch_size
    
    existing_ids = set(existing_ids)
    print(f" Found {len(existing_ids)} existing embeddings")
    print("=" * 50)
    
    return existing_ids

def find_image_path(uuid, trusted_objects):
    # Find image path for given UUID in trusted objects.
    for obj in trusted_objects:
        if obj.object_name.endswith(f"{uuid}.jpg"):
            return obj.object_name
    return None

def load_image_from_minio(client, trusted_bucket, image_path):
    # Load and convert image from MinIO to numpy array.
    try:
        image_data = client.get_object(trusted_bucket, image_path)
        image_bytes = image_data.read()
        image_data.close()
        image_data.release_conn()
        
        # Convert to PIL Image and then to numpy array
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        return np.array(image)
    except Exception as e:
        raise RuntimeError(f"Could not load image: {e}")

def create_metadata_dict(row):
    # Create metadata dictionary for ChromaDB.
    return {
        'uuid': row.get('uuid'),
        'kingdom': str(row.get('kingdom', '')),
        'phylum': str(row.get('phylum', '')),
        'class': str(row.get('class', '')),
        'order': str(row.get('order', '')),
        'family': str(row.get('family', '')),
        'genus': str(row.get('genus', '')),
        'species': str(row.get('species', '')),
        'scientific_name': str(row.get('scientific_name', '')),
        'common_name': str(row.get('common', '')),
        'combined_text': row['combined_text']
    }

def add_embeddings_to_collection(collection, uuid, image_array, text_description, metadata_dict, existing_ids):
    # Add image and text embeddings to ChromaDB collection.
    img_id = f"{uuid}_img"
    txt_id = f"{uuid}_txt"
    
    try:
        # Add image embedding
        collection.add(
            ids=[img_id],
            images=[image_array],
            metadatas=[{**metadata_dict, "modality": "image"}])
        
        existing_ids.add(img_id)
        print(f" Added image embedding for {uuid}")
        
        # Add text embedding
        text_doc = ensure_native_str(text_description)
        collection.add(
            ids=[txt_id],
            documents=[str(text_doc)],
            metadatas=[{**metadata_dict, "modality": "text"}])
        
        existing_ids.add(txt_id)
        print(f" Added text embedding for {uuid}")
        
        return True
    except Exception as e:
        print(f" Error adding {uuid} to collection: {e}")
        return False

def process_single_item(row, client, trusted_bucket, trusted_objects, collection, existing_ids):
    # Process a single metadata row and add embeddings.
    uuid = row.get('uuid')
    if not uuid:
        return False, "No UUID"
    
    # Skip if already exists
    img_id = f"{uuid}_img"
    txt_id = f"{uuid}_txt"
    
    if img_id in existing_ids or txt_id in existing_ids:
        print(f" Skipping duplicate UUID: {uuid}")
        return False, "Duplicate"
    
    # Find image path
    image_path = find_image_path(uuid, trusted_objects)
    if not image_path:
        print(f" Image not found for UUID: {uuid}")
        return False, "Image not found"
    
    # Load image
    try:
        image_array = load_image_from_minio(client, trusted_bucket, image_path)
    except Exception as e:
        print(f" Could not load image {uuid}: {e}")
        return False, "Image load error"
    
    # Create text description
    text_description = create_combined_text(row)
    text_description = ensure_native_str(text_description).strip()
    
    if not text_description or text_description == '.':
        print(f" ERROR: No valid metadata fields found for UUID: {uuid}")
        return False, "Invalid metadata"
    
    # Create metadata dictionary
    metadata_dict = create_metadata_dict(row)
    
    # Add embeddings to collection
    success = add_embeddings_to_collection(collection, uuid, image_array, text_description, metadata_dict, existing_ids)
    
    return success, "Success" if success else "Collection error"

def process_all_embeddings(metadata_df, client, trusted_bucket, collection, existing_ids):
    # Process all metadata rows and create embeddings.
    print(" Processing images and creating multimodal embeddings...")
    
    # Load all trusted objects
    print(" Pre-loading all trusted objects...")
    trusted_objects = list(client.list_objects(trusted_bucket, prefix="", recursive=True))
    print(f" Loaded {len(trusted_objects)} objects from trusted-zone")
    
    processed_count = 0
    skipped_count = 0
    error_count = 0
    
    # Process each metadata row
    for idx, row in metadata_df.iterrows():
        success, reason = process_single_item(row, client, trusted_bucket, trusted_objects, collection, existing_ids)
        
        if success:
            processed_count += 1
            if processed_count % 50 == 0:
                print(f" Processed {processed_count} items...")
        elif reason == "Duplicate":
            skipped_count += 1
        else:
            error_count += 1
    
    return processed_count, skipped_count, error_count

# ==============================
#        Configuration
# ==============================
def process_exploitation_multimodal(
    minio_endpoint = "localhost:9000",
    access_key = "admin",
    secret_key = "password123"):

    TRUSTED_BUCKET = "trusted-zone"
    # set the working directory
    try:
        SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__)) # in orchestrated
    except NameError:
        SCRIPT_DIR = os.getcwd() # in notebook
        
    CHROMA_DB = os.path.join(SCRIPT_DIR, "../exploitation_db")
    COLLECTION_NAME = "multimodal_embeddings"
    EMBEDDING_FUNCTION = "clip"
    
    print(" Starting Multimodal Embeddings Creation...")
    print(f" Collection: {COLLECTION_NAME}")
    print(f" Embedding Function: {EMBEDDING_FUNCTION}")
    print("=" * 50)
    
    # Setup MinIO and ChromaDB
    client = setup_minio_client(minio_endpoint, access_key, secret_key, TRUSTED_BUCKET)
    collection = setup_chromadb_collection(CHROMA_DB, COLLECTION_NAME)
    
    # Load and process metadata
    metadata_df = load_metadata_from_minio(client, TRUSTED_BUCKET)
    
    # Get existing embeddings
    existing_ids = get_existing_embeddings(collection)
    
    # Process all embeddings
    processed_count, skipped_count, error_count = process_all_embeddings(
        metadata_df, client, TRUSTED_BUCKET, collection, existing_ids)
    
    print("=" * 50)
    print(" Processing completed!")
    print(f" Processed: {processed_count} items")
    print(f" Skipped (duplicates): {skipped_count} items")
    print(f" Errors: {error_count} items")
    print("=" * 50)

process_exploitation_multimodal();

🚀 Starting Multimodal Embeddings Creation...
📊 Collection: multimodal_embeddings
🔧 Embedding Function: clip
 Connecting to MinIO...
✅ Connected to MinIO - Bucket 'trusted-zone' verified
🗄️ Setting up ChromaDB...
 Loading metadata from trusted-zone...
📄 Found 1 metadata files, using latest: metadata/trusted_metadata_2025_10_24_05_52_59.csv
✅ Downloaded metadata from metadata/trusted_metadata_2025_10_24_05_52_59.csv
📊 Loaded 6098 records from metadata
🧹 Cleaned up temporary metadata file
🗄️ Setting up ChromaDB collection...
📊 Found 9864 existing embeddings
🖼️ Processing images and creating multimodal embeddings...
🔍 Pre-loading all trusted objects...
📂 Loaded 6099 objects from trusted-zone
⏩ Skipping duplicate UUID: d0caebda-85d3-46fc-978e-58ca4c7d5cfd
⏩ Skipping duplicate UUID: cc3e8cd0-b993-4fdf-ae35-27880edf1597
⏩ Skipping duplicate UUID: bafa2323-81f5-4783-bb58-6b5d26a2e1b7
⏩ Skipping duplicate UUID: 9b074b8a-3fa2-4da5-9925-7f28ab1b9e36
⏩ Skipping duplicate UUID: 521e2087-5109-4c59-8